In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [1]:
S3_PATH_SPEAKER_VECTOR = {
    'vggvox-v1': {'model': 'v1/speaker-vector/pretrained-vggvox-v1.pb'},
    'vggvox-v2': {'model': 'v1/speaker-vector/pretrained-vggvox-v2.pb'},
    'deep-speaker': {'model': 'v1/speaker-vector/pretrained-deep-speaker.pb'},
    'speakernet': {'model': 'v1/speaker-vector/pretrained-speakernet.pb'},
}

In [3]:
for k in S3_PATH_SPEAKER_VECTOR.keys():
    if k not in ['multinomial']:
        print(k)
        os.system(f"wget https://f000.backblazeb2.com/file/malaya-speech-model/{S3_PATH_SPEAKER_VECTOR[k]['model']}")

vggvox-v1
vggvox-v2
deep-speaker
speakernet


In [4]:
import tensorflow as tf
from tensorflow.tools.graph_transforms import TransformGraph
from glob import glob
tf.set_random_seed(0)

In [7]:
pbs = glob('*.pb')
pbs

['pretrained-speakernet.pb',
 'pretrained-vggvox-v1.pb',
 'pretrained-vggvox-v2.pb',
 'pretrained-deep-speaker.pb']

In [6]:
!rm albert-base-sentiment.pb alxlnet-base-sentiment.pb

In [8]:
transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics, op=Dropout)',
             'fold_batch_norms',
             'fold_old_batch_norms',
             'quantize_weights(fallback_min=-10, fallback_max=10)',
             'strip_unused_nodes',
             'sort_by_execution_order']

for pb in pbs:
    input_graph_def = tf.GraphDef()
    with tf.gfile.FastGFile(pb, 'rb') as f:
        input_graph_def.ParseFromString(f.read())
        
    if 'speakernet' in pb:
        inputs = ['Placeholder', 'Placeholder_1']
    else:
        inputs = ['Placeholder']
    
    transformed_graph_def = TransformGraph(input_graph_def, 
                                           inputs,
                                           ['logits'], transforms)
    
    with tf.gfile.GFile(f'{pb}.quantized', 'wb') as f:
        f.write(transformed_graph_def.SerializeToString())

Instructions for updating:
Use tf.gfile.GFile.


In [11]:
quantized = glob('*.pb.quantized')
quantized

['pretrained-deep-speaker.pb.quantized',
 'pretrained-speakernet.pb.quantized',
 'pretrained-vggvox-v2.pb.quantized',
 'pretrained-vggvox-v1.pb.quantized']